In [7]:
import pandas as pd
from pyomo.environ import *

# Cargar el archivo Excel
archivo = r"C:\Users\Aminadab Cordova\Downloads\generator_unit_data.xlsx"  # Reemplázalo con el nombre real de tu archivo
df = pd.read_excel(archivo)

# Mostrar los primeros registros
print(df.head())

# Convertir la columna de zonas prohibidas a listas de tuplas
def parse_pz(value):
    """Convierte una cadena de zonas prohibidas en una lista de tuplas."""
    try:
        return [tuple(map(int, intervalo.strip("[]").split(','))) for intervalo in value.split(';')]
    except:
        return []  # En caso de valores vacíos o errores

df["PZ"] = df["PZ"].astype(str).apply(parse_pz)

# Mostrar la estructura de los datos procesados
print(df)


  Unit  Pmax  Pmin      ai    bi   ci  UR   DR   Pi                   PZ
0   G1   500   100  0.0070   7.0  240  80  120  440  [210,240];[350,380]
1   G2   200    50  0.0095  10.0  200  50   90  170   [90,110];[140,160]
2   G3   300    80  0.0090   8.5  220  65  100  200  [150,170];[210,240]
3   G4   150    50  0.0090  11.0  200  50   90  150    [80,90];[110,120]
4   G5   200    50  0.0080  10.5  220  50   90  190   [90,110];[140,150]
  Unit  Pmax  Pmin      ai    bi   ci  UR   DR   Pi                        PZ
0   G1   500   100  0.0070   7.0  240  80  120  440  [(210, 240), (350, 380)]
1   G2   200    50  0.0095  10.0  200  50   90  170   [(90, 110), (140, 160)]
2   G3   300    80  0.0090   8.5  220  65  100  200  [(150, 170), (210, 240)]
3   G4   150    50  0.0090  11.0  200  50   90  150    [(80, 90), (110, 120)]
4   G5   200    50  0.0080  10.5  220  50   90  190   [(90, 110), (140, 150)]
5   G6   120    50  0.0075  12.0  190  50   90  110    [(75, 85), (100, 105)]


In [10]:
# Crear modelo Pyomo
model = ConcreteModel()

# Definir conjuntos
model.Generadores = Set(initialize=df["Unit"].tolist())  # Conjunto de generadores

# Parámetros
model.Pmax = Param(model.Generadores, initialize=df.set_index("Unit")["Pmax"].to_dict())
model.Pmin = Param(model.Generadores, initialize=df.set_index("Unit")["Pmin"].to_dict())
model.a = Param(model.Generadores, initialize=df.set_index("Unit")["ai"].to_dict())
model.b = Param(model.Generadores, initialize=df.set_index("Unit")["bi"].to_dict())
model.c = Param(model.Generadores, initialize=df.set_index("Unit")["ci"].to_dict())
model.UR = Param(model.Generadores, initialize=df.set_index("Unit")["UR"].to_dict())
model.DR = Param(model.Generadores, initialize=df.set_index("Unit")["DR"].to_dict())

# Variables de potencia generada
model.Pg = Var(model.Generadores, within=NonNegativeReals)

# Función Objetivo: Minimizar el costo total de generación
def objetivo_rule(model):
    return sum(model.a[g] * model.Pg[g]**2 + model.b[g] * model.Pg[g] + model.c[g] for g in model.Generadores)

model.CostoTotal = Objective(rule=objetivo_rule, sense=minimize)

# Restricción de balance de potencia
Pd = 1000  # Demanda total del sistema
def balance_rule(model):
    return sum(model.Pg[g] for g in model.Generadores) == Pd

model.BalancePotencia = Constraint(rule=balance_rule)

# Restricción de límite inferior
def limite_inferior_rule(model, g):
    return model.Pmin[g] <= model.Pg[g]

model.LimiteInferior = Constraint(model.Generadores, rule=limite_inferior_rule)

# Restricción de límite superior
def limite_superior_rule(model, g):
    return model.Pg[g] <= model.Pmax[g]

model.LimiteSuperior = Constraint(model.Generadores, rule=limite_superior_rule)

# Restricciones de rampa
def rampas_rule(model, g):
    return (-model.DR[g] <= model.Pg[g] - df[df["Unit"] == g]["Pi"].values[0] <= model.UR[g])

model.LimiteRampa = Constraint(model.Generadores, rule=rampas_rule)

# Restricciones de zonas prohibidas
def zonas_prohibidas_rule(model, g):
    for zona in df[df["Unit"] == g]["PZ"].values[0]:  # Extraer zonas prohibidas
        model.add_constraint(Or(model.Pg[g] <= zona[0], model.Pg[g] >= zona[1]))

model.ZonasProhibidas = Constraint(model.Generadores, rule=zonas_prohibidas_rule)

# Resolver el modelo
solver = SolverFactory("glpk")
resultado = solver.solve(model, tee=True)

# Mostrar resultados
for g in model.Generadores:
    print(f"Generador {g}: {model.Pg[g].value:.2f} MW")
print(f"Costo Total: {model.CostoTotal():.2f} USD")

ERROR: Rule failed when generating expression for Constraint LimiteRampa with
index G1: PyomoException: Cannot convert non-constant Pyomo expression (-120
<=  Pg[G1] - 440) to bool. This error is usually caused by using a Var, unit,
or mutable Param in a Boolean context such as an "if" statement, or when
checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'LimiteRampa' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression (-120  <=
        Pg[G1] - 440) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    a

PyomoException: Cannot convert non-constant Pyomo expression (-120  <=  Pg[G1] - 440) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

SIN FILE

In [1]:
import pandas as pd
from pyomo.environ import *

# Load the data
df = pd.read_csv("generators.csv")

df

,Unit,Pmax (MW),Pmin (MW),ai ($/MW2h),bi ($/MWh),ci ($/h),URi (MW/h),DRi (MW/h),Pi (MW),Prohibited Zones
0,G1,500,100,0.0070,7.0,240,80,120,440,"[210,240];[350,380]"
1,G2,200,50,0.0095,10.0,200,50,90,170,"[90,110];[140,160]"
2,G3,300,80,0.0090,8.5,220,65,100,200,"[150,170];[210,240]"
3,G4,150,50,0.0090,11.0,200,50,90,150,"[80,90];[110,120]"
4,G5,200,50,0.0080,10.5,220,50,90,190,"[90,110];[140,150]"
5,G6,120,50,0.0075,12.0,190,50,90,110,"[75,85];[100,105]"


In [2]:
# Create a model
model = ConcreteModel()

# Define sets
model.G = Set(initialize=df.index.tolist())  # Generators

# Define parameters
model.Pmax = Param(model.G, initialize=df['Pmax (MW)'].to_dict())
model.Pmin = Param(model.G, initialize=df['Pmin (MW)'].to_dict())
model.a = Param(model.G, initialize=df['ai ($/MW2h)'].to_dict())
model.b = Param(model.G, initialize=df['bi ($/MWh)'].to_dict())
model.c = Param(model.G, initialize=df['ci ($/h)'].to_dict())
model.UR = Param(model.G, initialize=df['URi (MW/h)'].to_dict())
model.DR = Param(model.G, initialize=df['DRi (MW/h)'].to_dict())
model.demand = Param(initialize=300)  # Total demand (example value)

# Define variables
model.generation = Var(model.G, within=NonNegativeReals)

# Objective function: Minimize total cost
def total_cost(model):
    return sum(model.a[g] * model.generation[g]**2 + model.b[g] * model.generation[g] + model.c[g] for g in model.G)

model.obj = Objective(rule=total_cost, sense=minimize)

# Constraint: Meet the demand
def demand_constraint(model):
    return sum(model.generation[g] for g in model.G) == model.demand

model.demand_constraint = Constraint(rule=demand_constraint)

# Constraint: Generation limits
def generation_limits(model, g):
    return inequality(model.Pmin[g], model.generation[g], model.Pmax[g])

model.generation_limits = Constraint(model.G, rule=generation_limits)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print results
for g in model.G:
    print(f"Generation of {g}: {model.generation[g].value} units")
print(f"Total cost: {model.obj()} units")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.12.13\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Generation of 0: 77.57375805043431 units
Generation of 1: 39.98363155313737 units
Generation of 2: 62.53662428810198 units
Generation of 3: 39.97802219044455 units
Generation of 4: 39.98045549776971 units
Generation of 5: 39.97329523924506 units
Total cost: 4215.310997904946 units


In [4]:
import pandas as pd
from pyomo.environ import *

# Load the data
df = pd.read_csv("generators.csv")

# Create a model
model = ConcreteModel()

# Define sets
model.G = Set(initialize=df.index.tolist())  # Generators

# Define parameters
model.Pmax = Param(model.G, initialize=df['Pmax (MW)'].to_dict())
model.Pmin = Param(model.G, initialize=df['Pmin (MW)'].to_dict())
model.a = Param(model.G, initialize=df['ai ($/MW2h)'].to_dict())
model.b = Param(model.G, initialize=df['bi ($/MWh)'].to_dict())
model.c = Param(model.G, initialize=df['ci ($/h)'].to_dict())
model.UR = Param(model.G, initialize=df['URi (MW/h)'].to_dict())
model.DR = Param(model.G, initialize=df['DRi (MW/h)'].to_dict())
model.demand = Param(initialize=300)  # Total demand (example value)

# Define variables
model.generation = Var(model.G, within=NonNegativeReals)

# Objective function: Minimize total cost
def total_cost(model):
    return sum(model.a[g] * model.generation[g]**2 + model.b[g] * model.generation[g] + model.c[g] for g in model.G)

model.obj = Objective(rule=total_cost, sense=minimize)

# Constraint: Meet the demand with a tolerance
def demand_constraint(model):
    return sum(model.generation[g] for g in model.G) == model.demand

model.demand_constraint = Constraint(rule=demand_constraint)

# Constraint: Generation limits
def generation_limits(model, g):
    return inequality(model.Pmin[g], model.generation[g], model.Pmax[g])

model.generation_limits = Constraint(model.G, rule=generation_limits)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Check solver status
if results.solver.termination_condition != TerminationCondition.optimal:
    print("Warning: Solver did not find an optimal solution.")
    print(f"Termination condition: {results.solver.termination_condition}")
    print(f"Solver message: {results.solver.message}")

# Print results
for g in model.G:
    print(f"Generation of {g}: {model.generation[g].value} units")
print(f"Total cost: {model.obj()} units")

# Additional debug information
total_generation = sum(model.generation[g].value for g in model.G)
print(f"Total generation: {total_generation} units")
print(f"Demand: {model.demand.value} units")

# Check if total generation meets the demand
if total_generation < model.demand.value:
    print("Warning: Total generation is less than the demand.")
elif total_generation > model.demand.value:
    print("Warning: Total generation is more than the demand.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

CODIGO EJEMPLO

In [ ]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjunto de generadores
model.Generadores = Set(initialize=['G1', 'G2', 'G3', 'G4', 'G5', 'G6'])

# Parámetros de los generadores
datos_generadores = {
    'G1': {'Pmax': 500, 'Pmin': 100, 'ai': 0.007, 'bi': 7, 'ci': 240, 'UR': 80, 'DR': 120, 'Pi': 440},
    'G2': {'Pmax': 200, 'Pmin': 50, 'ai': 0.0095, 'bi': 10, 'ci': 200, 'UR': 50, 'DR': 90, 'Pi': 170},
    'G3': {'Pmax': 300, 'Pmin': 80, 'ai': 0.009, 'bi': 8.5, 'ci': 220, 'UR': 65, 'DR': 100, 'Pi': 200},
    'G4': {'Pmax': 150, 'Pmin': 50, 'ai': 0.009, 'bi': 11, 'ci': 200, 'UR': 50, 'DR': 90, 'Pi': 150},
    'G5': {'Pmax': 200, 'Pmin': 50, 'ai': 0.008, 'bi': 10.5, 'ci': 220, 'UR': 50, 'DR': 90, 'Pi': 190},
    'G6': {'Pmax': 120, 'Pmin': 50, 'ai': 0.0075, 'bi': 12, 'ci': 190, 'UR': 50, 'DR': 90, 'Pi': 110}
}

# Definir parámetros en el modelo
model.Pmax = Param(model.Generadores, initialize={g: datos_generadores[g]['Pmax'] for g in model.Generadores})
model.Pmin = Param(model.Generadores, initialize={g: datos_generadores[g]['Pmin'] for g in model.Generadores})
model.ai = Param(model.Generadores, initialize={g: datos_generadores[g]['ai'] for g in model.Generadores})
model.bi = Param(model.Generadores, initialize={g: datos_generadores[g]['bi'] for g in model.Generadores})
model.ci = Param(model.Generadores, initialize={g: datos_generadores[g]['ci'] for g in model.Generadores})
model.UR = Param(model.Generadores, initialize={g: datos_generadores[g]['UR'] for g in model.Generadores})
model.DR = Param(model.Generadores, initialize={g: datos_generadores[g]['DR'] for g in model.Generadores})
model.Pi = Param(model.Generadores, initialize={g: datos_generadores[g]['Pi'] for g in model.Generadores})

# Variable de generación de cada generador
model.Pg = Var(model.Generadores, within=NonNegativeReals)

# Restricciones de límites de generación
def limite_inferior_rule(model, g):
    return model.Pmin[g] <= model.Pg[g]

def limite_superior_rule(model, g):
    return model.Pg[g] <= model.Pmax[g]

model.LimiteInferior = Constraint(model.Generadores, rule=limite_inferior_rule)
model.LimiteSuperior = Constraint(model.Generadores, rule=limite_superior_rule)

# Restricciones de rampas
def rampa_subida_rule(model, g):
    return model.Pg[g] - model.Pi[g] <= model.UR[g]

def rampa_bajada_rule(model, g):
    return model.Pi[g] - model.Pg[g] <= model.DR[g]

model.RampaSubida = Constraint(model.Generadores, rule=rampa_subida_rule)
model.RampaBajada = Constraint(model.Generadores, rule=rampa_bajada_rule)

# Función objetivo: minimizar el costo de generación
def costo_total_rule(model):
    return sum(model.ai[g] * model.Pg[g] ** 2 + model.bi[g] * model.Pg[g] + model.ci[g] for g in model.Generadores)

model.CostoTotal = Objective(rule=costo_total_rule, sense=minimize)

# Resolver el modelo
solver = SolverFactory('glpk') # IPOPT no lineales
result = solver.solve(model)

# Mostrar resultados
model.pprint()


solver 'glpk'


ApplicationError: No executable found for solver 'glpk'